In [ ]:
# default_exp desc.metrics.main

# Exploration of your data

> This module comprises some of the statistical and inference techniques to describe the inner properties of software data. The submodules might include:
>
> - Descriptive statistics
> - Software Metrics
> - Information Theory
> - Learning Principels Detection (Occams' Razor, Biased data, and Data Snooping)
> - Inference: Probabilistic and Causal

### Specifically in this module

> - Cyclomatic complexity (CYCLO)
> - Number of lines of code (NLOC)
> - Lack of Cohesion of Methods 5 (LCOM5)

### Limitations of Tools

> Languages this module plans to be able to process: Java, Javascript, Python, C++, C, Scala
##### Current Limitations
> - CYCLO/NLOC
> > - Can measure: Java
> > - Cannot measure: Python
> > - Can possibly measure: C, C++, Scala, Javascript
> - LCOM5
> > - Can measure: Java
> > - Cannot measure: Python, C++, Scala, Javascript
> > - Undefined: C

In [ ]:
# #hide
# from nbdev.showdoc import *

In [ ]:
# !pip install metrics # Outdated Cyclomatic Complexity tool
!pip install lizard
!pip install tree_sitter

# The below modules need to be loaded when not being used in Google Colab
!pip install pandas
!pip install scipy
!pip install matplotlib
!pip install chardet
!pip install bs4

You should consider upgrading via the '/Users/wilsmccreight/Desktop/Research/ds4se/notebook-venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wilsmccreight/Desktop/Research/ds4se/notebook-venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wilsmccreight/Desktop/Research/ds4se/notebook-venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wilsmccreight/Desktop/Research/ds4se/notebook-venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wilsmccreight/Desktop/Research/ds4se/notebook-venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wilsmccreight/Desktop/Research/ds4se/notebook-venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wilsmccreight/Desktop/Research/ds4se/notebook-venv/bin/python3 -m 

In [ ]:
# export
# Imports
import pandas as pd
from numpy import mean, std
from statistics import median
from scipy.stats import sem, t
import lizard
import matplotlib.pyplot as plt
from tree_sitter import Language, Parser, Node
#Decoding files
import chardet
from bs4 import UnicodeDammit


# TODO: Remove when mongo call is implemented
import os


from desc_metrics_parser_java import ParserJava
from desc_metrics_parser_cpp import ParserCPP
from desc_metrics_parser_python import ParserPython

In [ ]:
!git clone https://github.com/tree-sitter/tree-sitter-java
!git clone https://github.com/tree-sitter/tree-sitter-cpp
!git clone https://github.com/tree-sitter/tree-sitter-python

fatal: destination path 'tree-sitter-java' already exists and is not an empty directory.
Cloning into 'tree-sitter-cpp'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 1327 (delta 44), reused 59 (delta 31), pack-reused 1244
Receiving objects: 100% (1327/1327), 38.36 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (862/862), done.
Cloning into 'tree-sitter-python'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 2135 (delta 40), reused 58 (delta 25), pack-reused 2051
Receiving objects: 100% (2135/2135), 17.91 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (1358/1358), done.


In [ ]:
# export
def get_unicode(file_path):
    """Detects file encoding and returns unicode. Inspired by http://reinvantveer.github.io/2017/05/19/unicode-dammit.html"""
    with open(file_path, 'rb') as f:
        detection = chardet.detect(f.read())
        
    enc = detection["encoding"]
    if detection["encoding"] == "ascii":
        with open(file_path, encoding="ascii") as f:
            data = f.read()
    elif detection["encoding"] == "ISO-8859-9":
        with open(file_path, encoding="utf-8") as f:
            enc = "utf-8"
            data = f.read()
    else:
        try:
            # Try to open as non unicode file
            with open(file_path, encoding=detection["encoding"]) as f:
                data = f.read()
        except Exception as e:
            raise ValueError(f"Cannot return dictionary from empty or invalid csv file {file_path} due to {e}")

    if not data:
        raise ValueError(f"Cannot return dictionary from empty or invalid csv file {file_path}")

    return UnicodeDammit(data).unicode_markup, enc

In [ ]:
def simulate_getting_dataframes_from_mongo(folder_path):
    """Loads files from a specified folder into a pandas dataframe"""
    corpus_data = {"system": [], "name": [], "ground_truth": [], "contents": [], "encoding": []}
    for file in os.listdir(folder_path):
        if not os.path.isdir(os.path.join(folder_path, file)) and file != ".DS_Store":
            corpus_data["system"].append(None)
            corpus_data["name"].append(file)
            corpus_data["ground_truth"].append("src")
            contents, enc = get_unicode(os.path.join(folder_path, file))
            corpus_data['encoding'].append(enc)
            corpus_data['contents'].append(contents)
    corpus_df = pd.DataFrame(data = corpus_data)
    return corpus_df

In [ ]:
# export
def add_mccabe_metrics(df, data_col, name_col):
    """Adds information about function length and cyclomatic complexity for classes to a dataframe"""
    num_funcs = []
    class_ccn = []
    avg_func_ccn = []
    avg_func_nloc = []
    for i in range(len(df)):
        file_num_funcs = []
        file_class_ccn = []
        file_avg_func_ccn = []
        file_avg_func_nloc = []
        metrics = lizard.analyze_file.analyze_source_code(df[name_col][i], df[data_col][i])
        class_dict = {}
        for func in metrics.function_list:
            class_name = '::'.join(func.name.split("::")[:-1])
            if class_name in class_dict:
                class_dict[class_name].append(func)
            else:
                class_dict[class_name] = [func]
        for class_key in class_dict:
            total_class_ccn = 0
            total_class_nloc = 0
            for func in class_dict[class_key]:
                total_class_ccn += func.cyclomatic_complexity
                total_class_nloc += func.length
            file_num_funcs.append(len(class_dict[class_key]))
            file_class_ccn.append(total_class_ccn)
            file_avg_func_ccn.append(total_class_ccn/len(class_dict[class_key]))
            file_avg_func_nloc.append(total_class_nloc/len(class_dict[class_key]))

        num_funcs.append(file_num_funcs)
        class_ccn.append(file_class_ccn)
        avg_func_ccn.append(file_avg_func_ccn)
        avg_func_nloc.append(file_avg_func_nloc)

    df["num_funcs"] = num_funcs
    df["class_ccn"] = class_ccn
    df["avg_func_ccn"] = avg_func_ccn
    df["avg_func_nloc"] = avg_func_nloc
    return df

In [ ]:
def create_parser_builds(path=None):
    """Creates a dictionary of tree-sitter parsers for select languages"""
    Language.build_library(
        # Store the library in the `build` directory
        'build/my-languages.so',

        # Include one or more languages
        [
            'tree-sitter-java',
            'tree-sitter-cpp',
            'tree-sitter-python'
        ]
    )

    JAVA_LANGUAGE = Language('build/my-languages.so', 'java')
    CPP_LANGUAGE = Language('build/my-languages.so', 'cpp')
    PY_LANGUAGE = Language('build/my-languages.so', 'python')
    
    return {"java":JAVA_LANGUAGE, "cpp":CPP_LANGUAGE, "py":PY_LANGUAGE}

In [ ]:
def lcom5_parser(ext):
    if ext == "java":
        return ParserJava()
    if ext == "cpp":
        return ParserCPP()
    if ext == "py":
        return ParserPython()
    print(f"ERROR: No LCOM5 parser available for .{ext} extension files")

In [ ]:
#export
def calculate_lcom5(tree, extension, file_bytes, name):
    """Parses the syntax tree of code to calculate the LCOM5 of its classes"""
    parser = lcom5_parser(extension)
    if not parser:
        return ["Undefined"]

    root_node = tree.root_node
    class_nodes = parser.find_class_nodes(root_node)
    class_method_names = []
    class_field_names = []
    class_dfc = [] # Distinct field calls, as per the definition of LCOM5
    for node in enumerate(class_nodes):
        class_method_names.append(parser.find_method_names(node[1], file_bytes))
        class_field_names.append(parser.find_field_names(node[1], class_method_names[node[0]], file_bytes))
        class_dfc.append(parser.distinct_field_calls(node[1], class_field_names[node[0]], file_bytes))
    lcom5_list = []
    for j in range(len(class_nodes)):
        num_fields = len(class_field_names[j])
        num_meths = len(class_method_names[j])
        num_dac = class_dfc[j]
        numerator = num_dac - (num_meths*num_fields)
        denominator = num_fields - (num_meths*num_fields)
        if denominator == 0:
            lcom5_list.append("Undefined")
        else:
            lcom5_list.append(numerator/denominator)
    return lcom5_list

In [ ]:
#export
def add_lcom5(df, col):
    """Adds a column with the LCOM5 of each class of each file to a dataframe"""
    lang_builds = create_parser_builds()
    parser = Parser()
    class_lcom5 = []
    
    for i in range(len(df)):
        ext = df["name"][i].split('.')[-1]
        parser.set_language(lang_builds[ext])
        enc = df["encoding"][i]
        tree = parser.parse(bytes(df["contents"][i], df["encoding"][i]))
        class_lcom5.append(calculate_lcom5(tree, ext, bytes(df["contents"][i], df["encoding"][i]), df["name"][i]))
    df["class_lcom5"] = class_lcom5
    
    return df

In [ ]:
#export
def flatten_lol(list_list):
    """Takes in a list of lists and flattens it, returning a list of each entry"""
    flattened_list = []
    for sublist in list_list:
        for entry in sublist:
            flattened_list.append(entry)
    return flattened_list

In [ ]:
# export
def display_numeric_col_stats(col, conf = 0.95, sig_figs = 4, clean=True, verbose_clean=False):
    """Computes statistical metrics about the entries in a dataframe column or list"""
    previous_length = len(col)
    numeric_types = [int, float, complex]
    if clean: col = [x for x in col if type(x) in numeric_types]
    if verbose_clean: print(f"Cleaning removed {previous_length - len(col)} non-numeric entries")

    if len(col) < 1:
        print("Error, data must contain at least one valid entry to display statistics")
        return

    print("Min =", round(min(col), sig_figs))
    print("Max =", round(max(col), sig_figs))
    print("Average =", round(mean(col), sig_figs))
    print("Median =", round(median(col), sig_figs))
    print("Standard Deviation =", round(std(col), sig_figs))
    
    n = len(col)
    m = mean(col)
    std_err = sem(col)
    h = std_err * t.ppf((1 + conf) / 2, n - 1)

    start = m - h
    end = m + h
    print(f"{conf} of data points fall between {round(start, sig_figs)} and {round(end, sig_figs)}")

In [ ]:
#export
def display_numeric_col_hist(col, col_name="Metric", num_bins=20, clean=True, verbose_clean=False):
    """Displays a histogram with a customized number of bins for the data in a specified dataframe column or list"""
    previous_length = len(col)
    numeric_types = [int, float, complex]
    if clean: col = [x for x in col if type(x) in numeric_types]
    if verbose_clean: print(f"Cleaning removed {previous_length - len(col)} non-numeric entries")

    if len(col) < 1:
        print("Error, data must contain at least one valid entry to display histogram")
        return    

    rng = max(col) - min(col)
    num = len(col)
    stnd_dev = std(col)

    plt.hist(col, num_bins, color="blue", alpha=0.5, edgecolor="black", linewidth=1.0)
    plt.title(col_name + " Histogram")
    plt.ylabel("Value  Range  Occurrences")
    plt.xlabel(col_name)
    plt.show()